In [11]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = (
    SparkSession
    .builder
    .config("spark.sql.warehouse.dir", "./temp")
    .master('local[*]')
    .getOrCreate()
)

In [18]:
df_transacoes = spark.read.csv(r"C:\dados-tcs-itau\Etapa 4\dados\transacoes.csv", header=True, inferSchema=True)
df_transacoes.show()
df_clientes = spark.read.csv(r"C:\dados-tcs-itau\Etapa 4\dados\clientes.csv", header=True, inferSchema=True)
df_clientes.show()

+------------+----------+-----+--------------+-----------+
|id_transacao|id_cliente|valor|data_transacao|  categoria|
+------------+----------+-----+--------------+-----------+
|           1|         1|100.0|    2023-11-01|    Compras|
|           2|         2|200.0|    2023-11-02|Alimentação|
|           3|         3|150.0|    2023-11-03|    Compras|
|           4|         4|300.0|    2023-11-01|     Viagem|
|           5|         5| 50.0|    2023-11-04|Alimentação|
+------------+----------+-----+--------------+-----------+

+----------+-----+-----+--------------+
|id_cliente| nome|idade|        cidade|
+----------+-----+-----+--------------+
|         1| João|   35|     São Paulo|
|         2|Maria|   25|Rio de Janeiro|
|         3| José|   40|      Curitiba|
|         4|  Ana|   28|     São Paulo|
|         5|Paulo|   50|Belo Horizonte|
+----------+-----+-----+--------------+



In [13]:
df_join = df_transacoes.join(df_clientes, on="id_cliente", how="inner")
df_join.show()

+----------+------------+-----+--------------+-----------+-----+-----+--------------+
|id_cliente|id_transacao|valor|data_transacao|  categoria| nome|idade|        cidade|
+----------+------------+-----+--------------+-----------+-----+-----+--------------+
|         1|           1|100.0|    2023-11-01|    Compras| João|   35|     São Paulo|
|         2|           2|200.0|    2023-11-02|Alimentação|Maria|   25|Rio de Janeiro|
|         3|           3|150.0|    2023-11-03|    Compras| José|   40|      Curitiba|
|         4|           4|300.0|    2023-11-01|     Viagem|  Ana|   28|     São Paulo|
|         5|           5| 50.0|    2023-11-04|Alimentação|Paulo|   50|Belo Horizonte|
+----------+------------+-----+--------------+-----------+-----+-----+--------------+



In [15]:
df_total_transacoes = df_transacoes.groupBy("id_cliente") \
    .agg(F.sum("valor").alias("total_transacoes"))
    
df_resultado = df_total_transacoes.join(df_clientes, on="id_cliente", how="inner")
    
df_resultado.show()

+----------+----------------+-----+-----+--------------+
|id_cliente|total_transacoes| nome|idade|        cidade|
+----------+----------------+-----+-----+--------------+
|         1|           100.0| João|   35|     São Paulo|
|         2|           200.0|Maria|   25|Rio de Janeiro|
|         3|           150.0| José|   40|      Curitiba|
|         4|           300.0|  Ana|   28|     São Paulo|
|         5|            50.0|Paulo|   50|Belo Horizonte|
+----------+----------------+-----+-----+--------------+



In [20]:
df_maior_cliente = df_total_transacoes.orderBy(F.col("total_transacoes").desc()).limit(1)

#CASO TENHA MAIS DE UM CLIENTE COM O MESMO VALOR MÁXIMO
df_maior_cliente = df_total_transacoes.filter(
    F.col("total_transacoes") == df_total_transacoes.agg(F.max("total_transacoes")).collect()[0][0]
)

df_resultado = df_maior_cliente.join(df_clientes, on="id_cliente", how="inner")

df_resultado.show()

+----------+----------------+----+-----+---------+
|id_cliente|total_transacoes|nome|idade|   cidade|
+----------+----------------+----+-----+---------+
|         4|           300.0| Ana|   28|São Paulo|
+----------+----------------+----+-----+---------+

